In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
data = pd.read_csv('data/train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.columns

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [5]:
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 
                       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 
                       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 
                       'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                       'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
                       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 
                       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 
                       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
                       'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'YrSold']

In [6]:
numerical_columns = ['LotFrontage', 'LotArea', 
                        'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
                        '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 
                        'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                        'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 
                        'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
                        'PoolArea', 'MiscVal', 'MoSold']

In [7]:
from bokeh.charts import Bar
from bokeh.charts import defaults
from bokeh.io import gridplot
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

In [8]:
def bar(df, x, y='SalePrice', title=None, width=280, height=280):
    df = pd.pivot_table(df, index=[x], values=[y], aggfunc=np.count_nonzero).reset_index()
    total_sum = df[y].sum()
    df['percent'] = df[y].apply(lambda i: "{0:.3}%".format(float(i*100)/total_sum))
    df[x] = df[x].astype('str')
    
    
    source = ColumnDataSource(data=dict(x=df[x], y=df[y], percent=df['percent']))
    hover = HoverTool(
        tooltips=[
            ('category', '@x'),
            ('percent', '@percent'),
        ]
    )
    p = figure(title=title, 
               x_range=df[x].values.tolist(), 
               width=width, height=height, 
               toolbar_location=None,
               tools=[hover]
              )
    
    p.vbar(x='x', source=source, width=0.5, bottom=0, top='y')
    p.yaxis.visible = False

    p.xaxis.axis_label = x
    p.xaxis.axis_label_standoff = -1
#     p.add_tools(hover)
    return p

In [10]:
plots = [bar(data, categorical_columns[col]) for col in xrange(0, len(categorical_columns))]
chunks = [plots[x:x+3] for x in xrange(0, len(plots), 3)]
show(gridplot(chunks, toolbar_location=None))

In [11]:
p = bar(data, 'MSSubClass', width=600, height=600)
show(p)

In [24]:
x = 'MSSubClass'
y = 'SalePrice'
df = pd.pivot_table(data, 
                    index=[x], 
                    values=[y], 
                    aggfunc=np.count_nonzero).reset_index().sort_values(
    by=['SalePrice'], ascending=True) 
total_sum = df[y].sum()
df['percent'] = df[y] * 100 / total_sum
# df['percent'] = df[y].apply(lambda i: "{0:.3}%".format(float(i*100)/total_sum))
df

,MSSubClass,SalePrice,percent
2,40,4,0.273973
13,180,10,0.684932
3,45,12,0.821918
7,75,16,1.095890
9,85,20,1.369863
14,190,30,2.054795
10,90,52,3.561644
8,80,58,3.972603
6,70,60,4.109589
12,160,63,4.315068


In [26]:
df[df['percent'] < 2.0]

,MSSubClass,SalePrice,percent
2,40,4,0.273973
13,180,10,0.684932
3,45,12,0.821918
7,75,16,1.095890
9,85,20,1.369863


In [34]:
def get_pivoted(df, x, y):
    df = pd.pivot_table(data, 
                        index=[x], 
                        values=[y], 
                        aggfunc=np.count_nonzero).reset_index().sort_values(
        by=[y], ascending=True) 
    total_sum = df[y].sum()
    df['percent'] = df[y] * 100 / total_sum
    return df

In [118]:
def relevant_categories(df, x, y='SalePrice', threshold=2.0):
    pivoted = get_pivoted(df, x, y)
    pivoted = pivoted[pivoted['percent'] >= threshold]
#     df[x].apply(lambda x: str(x) if x in pivoted[x] else 'None')
#     relevant_categories = pivoted[x]
    df.loc[~a[x].isin(pivoted[x]), x] = None
#     return pivoted[x]
    return df
    

In [119]:
a = relevant_categories(data, 'MSSubClass')

In [120]:
a['MSSubClass'].value_counts()

20.0     536
60.0     299
50.0     144
120.0     87
30.0      69
160.0     63
0.0       62
70.0      60
80.0      58
90.0      52
190.0     30
Name: MSSubClass, dtype: int64

In [ ]:
transformed_data = data[numerical_columns].copy()

In [ ]:
for column in transformed_data[numerical_columns]:
    if transformed_data[column].isnull().values.any():
        print column

In [ ]:
transformed_data['LotFrontage'] = transformed_data['LotFrontage'].fillna(0)
transformed_data['MasVnrArea'] = transformed_data['MasVnrArea'].fillna(0)
transformed_data['GarageYrBlt'] = transformed_data['GarageYrBlt'].fillna(0)

In [ ]:
for column in categorical_columns:
    if data[column].isnull().values.any():
        print column

In [ ]:
for column in categorical_columns:
    if data[column].isnull().values.any():
        data[column] = data[column].fillna('None')

In [ ]:
transformed_data = pd.concat([transformed_data, data[categorical_columns].apply(LabelEncoder().fit_transform)],
                             axis=1)

In [ ]:
transformed_data

In [ ]:
transformed_data['Age'] = data['YrSold'] - data['YearBuilt']

In [ ]:
transformed_data['remodAge'] = data['YrSold'] - data['YearRemodAdd']

In [ ]:
transformed_data.columns

In [ ]:
# sns.pairplot(transformed_data[numerical_columns])

In [ ]:
for column in numerical_columns:
    plt.figure(column)
    sns.distplot(transformed_data[column]);

In [ ]:
from scipy.stats import boxcox, skew
from sklearn.preprocessing import StandardScaler

In [ ]:
for column in numerical_columns:
    print skew(transformed_data[column])

In [ ]:
for column in numerical_columns:
    if skew(transformed_data[column]) > 0.25:
        transformed_data[column], lam = boxcox(transformed_data[column]+1.00)

In [ ]:
for column in numerical_columns:
    plt.figure(column)
    sns.distplot(transformed_data[column]);

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()
transformed_data[numerical_columns] = ss.fit_transform(transformed_data[numerical_columns].values)
transformed_data[numerical_columns] = pd.DataFrame(transformed_data[numerical_columns],
                                                   index=transformed_data[numerical_columns].index,
                                                   columns=transformed_data[numerical_columns].columns)

In [ ]:
# ss = StandardScaler()

# for column in numerical_columns:
#     transformed_data[column] = ss.fit_transform(transformed_data[column].values.reshape(-1,1).flatten())

In [ ]:
for column in numerical_columns:
    plt.figure(column)
    sns.distplot(transformed_data[column]);

In [ ]:
corr_matrix = transformed_data[numerical_columns].corr()

In [ ]:
plt.subplots(figsize=(12, 9))
sns.heatmap(corr_matrix, vmax=.8, square=True)

In [ ]:
corr_matrix.values.shape

In [ ]:
corr_matrix.iloc[0, 2]

In [ ]:
for i in xrange(0, corr_matrix.values.shape[0]):
    for j in xrange(0, corr_matrix.values.shape[1]):
        if i < j:
            if corr_matrix.iloc[i, j] > 0.8:
                print i, j, corr_matrix.iloc[i, j]

### What to do with high correlated variables?

In [ ]:
from sklearn.cross_validation import _BaseKFold
from sklearn.utils.validation import check_random_state
import numpy as np

class RegStratifiedKFold(_BaseKFold):
    def __init__(self, y, n_folds=5, shuffle=False, random_state=None):
        super(RegStratifiedKFold, self).__init__(len(y), n_folds=n_folds, shuffle=shuffle, random_state=random_state)
        ylen = len(y)
        if ylen / n_folds <= 1:
            print "Too few elements in y. Still in ToDo list"
            exit()
        y_index = np.arange(ylen)[np.argsort(y)]
        num_classes = ylen / n_folds + int(ylen % n_folds != 0)
        
        num_head_tail_classes = n_folds * (ylen / n_folds / 2)
        head_classes = y_index[0:num_head_tail_classes].reshape(-1,n_folds)
        tail_classes = y_index[-num_head_tail_classes:].reshape(-1,n_folds)
        
        middle_class = y_index[num_head_tail_classes:-num_head_tail_classes] 
        middle_class = np.hstack([middle_class, -np.ones(n_folds - len(middle_class) % n_folds, dtype=int)])
        middle_class = middle_class.reshape(-1,n_folds)
        
        test_masks = np.vstack([head_classes, middle_class, tail_classes])
        self._test_masks = []
        
        if shuffle:
            rng = check_random_state(self.random_state)
            for cls in test_masks:
                rng.shuffle(cls)
                self._test_masks.append(cls.tolist())
        else:
            self._test_masks = test_masks
            
        self._test_masks = np.array(self._test_masks).T
        self._test_masks = [y[y!=-1] for y in self._test_masks]
        
    def _iter_test_masks(self):
        indarr = np.zeros( self.n, dtype = bool)
        for mask in self._test_masks:
            indarr[:] = False
            indarr[mask] = True 
            yield indarr

In [ ]:
def CV(target):
#     return StratifiedKFold(target, n_folds=3,
#                            shuffle=True, random_state=41)
    return RegStratifiedKFold(target, n_folds=5, shuffle=True, random_state=41)


def grid_search(model, params, cv):
    return GridSearchCV(estimator=model,
                        param_grid=params,
                        cv=cv)


def regression_results(model, x_true, y_true, y_pred):
    ret = {}
    ret["Explained variance regression score"] = explained_variance_score(
        y_true, y_pred, multioutput='uniform_average')
    ret["Mean Absolute Error"] = mean_absolute_error(
        y_true, y_pred)
    ret["Model socre"] = model.score(
        x_true, y_true)
    return ret


def draw_feature_importance(model, data):
    importances = model.feature_importances_
    std = np.std(
        [tree.feature_importances_ for tree in model.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    feature_names = data.columns
    # Plot the feature importances of the model
    plt.figure(figsize=(20, 15))
    plt.title("Feature importances")
    plt.bar(range(data.shape[1]), importances[indices],
            color="r", yerr=std[indices], align="center")
    plt.xticks(range(data.shape[1]), feature_names[indices], rotation=90)
    plt.xlim([-1, data.shape[1]])
    return plt


def model_test_metrics(model, data, target):
    ret = {}
    ret["Test Data score"] = model.fit(data, target).score(data, target)
    return merge_dicts(ret, regression_results(target, model.predict(data)))


def evaluate_model(model, data, target,
                   params=None, draw_features=True, verbose=True):
    train_data, test_data, train_target, test_target = train_test_split(
        data, target)
    cv=CV(train_target)
#     cv = 3
    ret = {}
    if params:
        grid = grid_search(model, params, cv)

        grid.fit(train_data, train_target)
        model = grid.best_estimator_
    else:
        model.fit(train_data, train_target)

    s = cross_val_score(model, train_data, train_target, cv=cv, n_jobs=-1)
    ret["Mean cross validation score"] = s.mean()
    predictions = model.predict(test_data)
    ret = merge_dicts(ret, regression_results(
        model, test_data, test_target, predictions))

    if draw_features:
        draw_feature_importance(model, train_data)
    return model, ret

def print_model_report(report, score=None, model=None):
    if model:
        print model, "\n"
    for title, value in report.iteritems():
        print "{:<50}{}".format(title, value)
    if score:
        print "\n", "{:<50}{}".format('score for test data', score)


def plot_predicted_measured(ax, y_real, predictions, fontsize=24, title=''):
    ax.scatter(y_real, predictions)
    ax.plot([predictions.min(), predictions.max()],
            [predictions.min(), predictions.max()], 'k--', lw=2)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    ax.set_title(title, fontsize=fontsize)


def set_style():
    plt.style.use(['seaborn-white', 'seaborn-paper'])


def plot_comparisions(y_real, y_lr, y_rd, y_rf, y_et):
    set_style()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
    fig.set_figheight(12)
    fig.set_figwidth(12)
    plot_predicted_measured(
        ax1, y_real, y_lr, fontsize=16, title='Linear Regression')
    plot_predicted_measured(
        ax2, y_real, y_rd, fontsize=16, title='RidgeCV')
    plot_predicted_measured(
        ax3, y_real, y_rf, fontsize=16, title='Random Forest')
    plot_predicted_measured(
        ax4, y_real, y_et, fontsize=16, title='Extra Trees')

    plt.show()

In [ ]:
def extra_trees(data, target, draw_features=False):
    params = {
        'n_estimators': [10, 20, 50, 100, 200],
        'max_depth': [1, 2, 3, 4, 5],
        'min_samples_split': [2, 10, 25, 50, 100],
    }
    return evaluate_model(ExtraTreesRegressor(n_jobs=-1),
                          data, target,
#                           params=params, 
                          draw_features=draw_features)

In [ ]:
x = transformed_data.copy()
y = data.SalePrice

In [ ]:
pd.pivot_table(data[data['SaleCondition'] == 'Abnorml'], index=['YrSold'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['MSSubClass'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['MSZoning'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Street'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Alley'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['LotShape'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['LandContour'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Utilities'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['LotConfig'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Neighborhood'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Condition1'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['Condition2'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['BldgType'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['HouseStyle'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['OverallQual'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['OverallCond'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['RoofStyle'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['RoofMatl'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')

In [ ]:
pd.pivot_table(data, index=['RoofMatl'], values=['SalePrice'], aggfunc=np.count_nonzero).plot(kind='bar')